In [ ]:
!pip install --q transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.6 MB/s eta 0:00:00


In [ ]:
dataset = load_dataset("RussianNLP/tape", dataset_name)

df_train = dataset['train'].data.to_pandas()
df_test = dataset['test'].data.to_pandas()

train_nli = Dataset.from_pandas(transform_df_to_mnli(df_train), split="train")

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(example):
    return tokenizer(example['premise'], example['hypothesis'], truncation=True, padding='max_length', max_length=300, return_tensors='pt').to(device)

train_dataset = train_nli.map(preprocess_function, batched=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import transformers
import torch
from sklearn.metrics import accuracy_score
from transformers import XLMRobertaForSequenceClassification, AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'

In [ ]:
def get_episode_data(data: pd.DataFrame, episode: int) -> pd.DataFrame:
    """
    Returns all the data from the specified episode
    Parameters
    ----------
    data: pd.DataFrame
        data to work with
    episode: int
        episode number
    Returns
    -------
    pd.DataFrame
        train data from the passed episode
    """
    ids = data.episode.apply(lambda x: episode in x)

    return data[ids]

In [ ]:
metric = evaluate.load("accuracy")


def get_hypotheses(question: list, prompt: str):
  _, ans_A, ans_B, ans_C, ans_D = question
  hypotheses = [prompt.replace('[ANSWER]', ans) for ans in [ans_A, ans_B, ans_C, ans_D]]
  return hypotheses

def get_label(index: int, answer: int):
  if index % 4 == answer:
    return 1
  else:
    return 0

def transform_df_to_mnli(df: pd.DataFrame):
  PROMPT = 'Ответ: [ANSWER]'

  # divide question text into the question and the answers
  df['question'] = df.question.replace(to_replace=r'\((A|B|C|D)\)', value='[SEP]', regex=True).apply(lambda x: x.split('[SEP]'))
  df['premise'] = 'Вопрос: ' + df.question.apply(lambda x: x[0] + f' Варианты ответа: (A) {x[1]} (B) {x[2]} (C) {x[3]} (D) {x[4]}') 
  df['hypothesis'] =  df.apply(lambda x: get_hypotheses(question=x['question'], prompt=PROMPT), axis=1)
  df = df.explode('hypothesis')

  # turn answers to binary labels
  df['answer'] = df['answer'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3})
  df.reset_index(inplace=True, drop=True)
  df['label'] = df.apply(lambda x: get_label(x.name, x['answer']), axis=1)
  
  return df[['premise', 'hypothesis', 'label']]


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def transform_predictions(pred, df):
    answers = pred.predictions[:,1].reshape(len(df), 4).argmax(axis=1)
    ans_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    answers = [ans_dict[ans] for ans in answers]

    assert len(answers) == len(df)

    return answers

            
def run_experiment(model_name, dataset_name, device):
    dataset = load_dataset("RussianNLP/tape", dataset_name)

    df_train = dataset['train'].data.to_pandas()
    df_test = dataset['test'].data.to_pandas()

    train_nli = Dataset.from_pandas(transform_df_to_mnli(df_train), split="train")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def preprocess_function(example):
        return tokenizer(example['premise'], example['hypothesis'], truncation=True, padding='max_length', max_length=300, return_tensors='pt').to(device)
    
    train_dataset = train_nli.map(preprocess_function, batched=True)

    evaluation_results = []
    episodes = list(np.unique(np.hstack(df_train.episode.values)))

    for episode in sorted(episodes):
        print('Training for episode ', str(episode))

        if 'roberta' in model_name:
            nli_model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
        else:
            nli_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
            
        k_shots = get_episode_data(df_train, episode)

        training_args = TrainingArguments(
            output_dir=f'.tape/{model_name}/results',
            evaluation_strategy='no',
            save_strategy='no',
            learning_rate=2e-5,
            per_device_train_batch_size=4,
            num_train_epochs=5,
            weight_decay=0.01,
            push_to_hub=False,
            logging_dir=f'.tape/{model_name}/logs',
            logging_steps=500,
            load_best_model_at_end=True,
            metric_for_best_model='accuracy',
            report_to='none'
            )

        trainer = Trainer(
            model=nli_model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=train_dataset,
            tokenizer=tokenizer,
            compute_metrics = compute_metrics
        )

        if episode != 4:
            trainer.train()

        train_pred = trainer.predict(train_dataset)
        train_answers = transform_predictions(train_pred, df_train)
        train_acc = accuracy_score(df_train.answer, train_answers)
        print('Episode: ', episode, 'Shot: ', k_shots.shape[0], 'Train Accuracy: ', train_acc)
        
        for perturbation, test in df_test.groupby('perturbation'):
            test_sample_nli = Dataset.from_pandas(transform_df_to_mnli(test), split="test")
            test_sample_dataset = test_sample_nli.map(preprocess_function, batched=True)

            test_pred = trainer.predict(test_sample_dataset)
            predictions = transform_predictions(test_pred, test)
            
            evaluation_results.append({
                "episode": episode,
                "shot": k_shots.shape[0],
                "slice": perturbation,
                "preds": predictions
            })

        inter_res = pd.DataFrame(evaluation_results)
        if 'roberta' in model_name:
          inter_res[inter_res.episode == episode].to_csv(f"drive/My Drive/Colab Notebooks/dl4nlp_labs/project/eval_results/{model_name}_{dataset_name.split('.')[0]}_{episode}.csv")
        else:
          inter_res[inter_res.episode == episode].to_csv(f"drive/My Drive/Colab Notebooks/dl4nlp_labs/project/eval_results/{model_name.split('/')[1]}_{dataset_name.split('.')[0]}_{episode}.csv")
        
        del trainer, nli_model
            
    return evaluation_results

In [ ]:
evaluation_results = run_experiment('DeepPavlov/rubert-base-cased', 'ru_openbook.episodes', device)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset tape downloaded and prepared to /root/.cache/huggingface/datasets/RussianNLP___tape/ru_openbook.episodes/0.0.1/df7620d31cfdf6508b0b8442f699b93aad2bdd0dac90800435e8f42628280577. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Episode:  17 Shot:  8 Train Accuracy:  0.7083333333333334


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
def create_submission(model: str, dataset: str):
  submission_df = pd.DataFrame()
  for ep in range(4,20):
    pred_df = pd.read_csv(f'./eval_results/{model}_{dataset}_{ep}.csv')
    submission_df = submission_df.append(pred_df)

  submission_df.drop(columns=['Unnamed: 0'], inplace=True)
  submission_df['preds'] = submission_df.preds.apply(eval)

  return submission_df

In [ ]:
create_submission('rubert-base-cased-nli-twoway', 'ru_worldtree').to_json(
          "./rubert_nli_submission/predictions/RuWorldTree.json",
          orient="records",
          force_ascii=False,
      )

create_submission('rubert-base-cased-nli-twoway', 'ru_openbook').to_json(
          "./rubert_nli_submission/predictions/RuOpenBookQA.json",
          orient="records",
          force_ascii=False,
      )

create_submission('rubert-base-cased', 'ru_worldtree').to_json(
          "./rubert_submission/predictions/RuWorldTree.json",
          orient="records",
          force_ascii=False,
      )

create_submission('rubert-base-cased', 'ru_openbook').to_json(
          "./rubert_submission/predictions/RuOpenBookQA.json",
          orient="records",
          force_ascii=False,
      )

create_submission('xlm-roberta-base', 'ru_worldtree').to_json(
          "./xlm-r_submission/predictions/RuWorldTree.json",
          orient="records",
          force_ascii=False,
      )

create_submission('xlm-roberta-base', 'ru_openbook').to_json(
          "./xlm-r_submission/predictions/RuOpenBookQA.json",
          orient="records",
          force_ascii=False,
      )